# Vicon CSV → Quantity of Motion (QdM)

This notebook computes:
- 3D marker trajectories from Vicon CSV exports
- Quantity of Motion (QdM) for wrists and head
- Exports results to an Excel summary file

Author: Matys Précloux  
Project: SYNCOGEST

In [6]:
import sys
from pathlib import Path
import pandas as pd

from src_vicon_utils import read_vicon_csv, find_xyz_cols, motion_quantity_point

In [7]:

# Project root + import path

PROJECT_ROOT = Path.cwd()
if PROJECT_ROOT.name == "notebooks":
    PROJECT_ROOT = PROJECT_ROOT.parent
sys.path.insert(0, str(PROJECT_ROOT))


# Mode (test vs raw) + paths

MODE = "raw"   # change to "raw" for full dataset

ROOT = PROJECT_ROOT / "data" / MODE / "vicon_csv"
RESULTS_DIR = PROJECT_ROOT / "results" / MODE
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

print("Mode:", MODE)
print("Project root:", PROJECT_ROOT)
print("Vicon CSV root:", ROOT)
print("ROOT exists:", ROOT.exists())
print("Results dir:", RESULTS_DIR.resolve())

Mode: raw
Project root: /Users/matysprecloux/Desktop/Master IEAP/Code MOTTET/Defense /SYNCOGESTM2
Vicon CSV root: /Users/matysprecloux/Desktop/Master IEAP/Code MOTTET/Defense /SYNCOGESTM2/data/raw/vicon_csv
ROOT exists: True
Results dir: /Users/matysprecloux/Desktop/Master IEAP/Code MOTTET/Defense /SYNCOGESTM2/results/raw


In [8]:

# 4) Process one CSV file

def process_one_csv(csv_path: Path):

    df = read_vicon_csv(csv_path)
    cols = list(df.columns)

    out = {"csv": str(csv_path), "file": csv_path.name}

    subjects = {
        "P1": {
            "WR_D": "poignet_D",
            "WR_G": "poignet_G",
            "TP_D": "Tempe_D",
            "TP_G": "Tempe_G",
        },
        "P2": {
            "WR_D": "2poignet_D",
            "WR_G": "2poignet_G",
            "TP_D": "2Tempe_D",
            "TP_G": "2Temps_G",
        }
    }

    for pid, tok in subjects.items():

        # ---- Wrists ----
        for side_key in ["WR_D", "WR_G"]:
            token = tok[side_key]
            X, Y, Z = find_xyz_cols(cols, token)

            if None in (X, Y, Z):
                out[f"{pid}_{token}_error"] = "missing X/Y/Z"
            else:
                q, nsteps = motion_quantity_point(df, X, Y, Z)
                out[f"{pid}_{token}_QDM_mm"] = q
                out[f"{pid}_{token}_nsteps"] = nsteps

        wd = out.get(f"{pid}_{tok['WR_D']}_QDM_mm")
        wg = out.get(f"{pid}_{tok['WR_G']}_QDM_mm")

        if wd is not None and wg is not None:
            out[f"{pid}_QDM_WRISTS_mm"] = wd + wg

        # ---- Head (temples) ----
        for side_key in ["TP_D", "TP_G"]:
            token = tok[side_key]
            X, Y, Z = find_xyz_cols(cols, token)

            if None in (X, Y, Z):
                out[f"{pid}_{token}_error"] = "missing X/Y/Z"
            else:
                q, nsteps = motion_quantity_point(df, X, Y, Z)
                out[f"{pid}_{token}_QDM_mm"] = q
                out[f"{pid}_{token}_nsteps"] = nsteps

        td = out.get(f"{pid}_{tok['TP_D']}_QDM_mm")
        tg = out.get(f"{pid}_{tok['TP_G']}_QDM_mm")

        if td is not None and tg is not None:
            out[f"{pid}_QDM_HEAD_mm"] = td + tg

    return out

In [9]:

# 5) Batch processing

csv_files = sorted(ROOT.rglob("*.csv"))
print("CSV found:", len(csv_files))

rows = []

for f in csv_files:
    try:
        rows.append(process_one_csv(f))
    except Exception as e:
        rows.append({"csv": str(f), "file": f.name, "error": repr(e)})

df_out = pd.DataFrame(rows)

out_path = RESULTS_DIR / "vicon_QDM_wrists_head_mm.xlsx"
df_out.to_excel(out_path, index=False)

print("Saved:", out_path.resolve())
df_out.head()

CSV found: 36
Saved: /Users/matysprecloux/Desktop/Master IEAP/Code MOTTET/Defense /SYNCOGESTM2/results/raw/vicon_QDM_wrists_head_mm.xlsx


,csv,file,P1_poignet_D_QDM_mm,P1_poignet_D_nsteps,P1_poignet_G_QDM_mm,P1_poignet_G_nsteps,P1_QDM_WRISTS_mm,P1_Tempe_D_QDM_mm,P1_Tempe_D_nsteps,P1_Tempe_G_QDM_mm,...,P2_2poignet_D_QDM_mm,P2_2poignet_D_nsteps,P2_2poignet_G_QDM_mm,P2_2poignet_G_nsteps,P2_QDM_WRISTS_mm,P2_2Tempe_D_QDM_mm,P2_2Tempe_D_nsteps,P2_2Temps_G_QDM_mm,P2_2Temps_G_nsteps,P2_QDM_HEAD_mm
0,/Users/matysprecloux/Desktop/Master IEAP/Code ...,SEATEDD01.csv,11257.605164,18011,5086.439843,18011,16344.045006,15748.775325,18011,18437.193954,...,6990.478950,18011,4791.810419,18011,11782.289369,10070.014078,18011,10379.521199,18011,20449.535277
1,/Users/matysprecloux/Desktop/Master IEAP/Code ...,SEATEDD02.csv,15422.894012,17999,16779.292373,17999,32202.186385,9725.644869,17999,10354.212811,...,16420.189086,17999,17622.889557,17999,34043.078643,9851.813433,17999,10217.118782,17999,20068.932215
2,/Users/matysprecloux/Desktop/Master IEAP/Code ...,SEATEDD03.csv,34870.992267,17999,27170.831678,17999,62041.823945,14105.980148,17999,14473.346651,...,19508.827550,17999,6708.991777,17999,26217.819326,8916.973687,17999,9655.490833,17999,18572.464520
3,/Users/matysprecloux/Desktop/Master IEAP/Code ...,SEATEDD04.csv,2661.923274,17999,3833.346976,17999,6495.270250,4847.753225,17999,4768.206439,...,7680.839058,17999,11250.402281,17999,18931.241339,7442.981553,17999,7080.281923,17999,14523.263476
4,/Users/matysprecloux/Desktop/Master IEAP/Code ...,SEATEDD05.csv,6273.538429,17999,6588.362586,17999,12861.901015,4558.721874,17999,4829.197593,...,2684.099411,17999,5080.767751,17999,7764.867162,5805.341913,17999,5511.189012,17999,11316.530925
